Librerías

In [1]:
import pandas as pd
import json
import chardet
import ast
from pandas import json_normalize
import pickle
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

De formaton JSON a CSV

In [3]:
#Se busca el ecoding del archivo 'australian_user_reviews.json'
with open(r'C:\Users\Nahir\Desktop\JSON\australian_user_reviews.json', 'rb') as file:
    rawdata = file.read()
    result = chardet.detect(rawdata)
print(result['encoding'])

MacRoman


In [ ]:
#Se leen los datos del archivo 'australian_user_reviews.json'
rows = []
with open(r'C:\Users\Nahir\Desktop\JSON\australian_user_reviews.json', encoding='MacRoman') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))
# Se convierte el contenido a un dataframe
df_reviews = pd.DataFrame(rows)
# Se guarda en formato CSV
df_reviews.to_csv('australian_user_reviews.csv', index=False)

Explorando los datos

In [4]:
#Se define el df rewievs.
df_r=pd.read_csv(r'C:\Users\Nahir\Desktop\CSV\australian_user_reviews.csv',low_memory=False)
df_r.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [57]:
#Se explora el contenido del df.
df_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [60]:
#Se calcula la cantidad de duplicados del df.
df_r.duplicated().sum()

313

In [61]:
#Se eleiminan los duplicados ya que es información redundante.
df_r.drop_duplicates(inplace=True)

In [62]:
#Se conoce estructura de la columna 'reviews'.
df_r["reviews"]

0        [{'funny': '', 'posted': 'Posted November 5, 2...
1        [{'funny': '', 'posted': 'Posted June 24, 2014...
2        [{'funny': '', 'posted': 'Posted February 3.',...
3        [{'funny': '', 'posted': 'Posted October 14, 2...
4        [{'funny': '3 people found this review funny',...
                               ...                        
25794    [{'funny': '', 'posted': 'Posted May 31.', 'la...
25795    [{'funny': '', 'posted': 'Posted June 17.', 'l...
25796    [{'funny': '1 person found this review funny',...
25797    [{'funny': '', 'posted': 'Posted July 21.', 'l...
25798    [{'funny': '1 person found this review funny',...
Name: reviews, Length: 25486, dtype: object

Debido a que la columna 'reviews' es una lista de diccionarios, procedemos a crear una nueva columna por cada llave y unirlas al df existente.

In [63]:
#Se convierten los strings a diccionarios.
df_r['reviews'] = df_r['reviews'].apply(ast.literal_eval)
#Se extrae la información de los diccionarios en la columna 'reviews'.
df_r_r = json_normalize(df_r['reviews'].apply(lambda x: x[0] if x else {}))
#Se concatenan las columnas desglosadas con el DataFrame original.
df_r2 = pd.concat([df_r[['user_id', 'user_url']], df_r_r], axis=1)

#Se verifica el nuevo DataFrame con las columnas desglosadas y las columnas originales.
df_r2.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,,Posted February 3.,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,,"Posted October 14, 2013.",,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,"Posted April 15, 2014.",,211420,35 of 43 people (81%) found this review helpful,True,Git gud


In [64]:
#Se explora la composición del df con las nuevas columnas agregadas.
df_r2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25799 entries, 0 to 18309
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      25486 non-null  object
 1   user_url     25486 non-null  object
 2   funny        25458 non-null  object
 3   posted       25458 non-null  object
 4   last_edited  25458 non-null  object
 5   item_id      25458 non-null  object
 6   helpful      25458 non-null  object
 7   recommend    25458 non-null  object
 8   review       25458 non-null  object
dtypes: object(9)
memory usage: 2.0+ MB


In [14]:
#Se verifica el tipo de dato de 'review'
print(type(df_r2['review']))

<class 'pandas.core.series.Series'>


In [15]:
#Se visualiza la columna 'review'
df_r2['review']

0        Simple yet with great replayability. In my opi...
1        I know what you think when you see this title ...
2        A suitably punishing roguelike platformer.  Wi...
3        This game... is so fun. The fight sequences ha...
4                                                  Git gud
                               ...                        
25794    I cried in the end its so sadding ]'; I wish l...
25795    Gra naprawdƒô fajna.Ale jest kilka rzeczy do k...
25796                                            Well Done
25797    this is a very fun and nice 80s themed shooter...
25798    had so much fun plaing this and collecting res...
Name: review, Length: 25799, dtype: object

Modificación de 'posted'

In [65]:
#Se elimina "Posted", comas y espacios, y se convierte a formato de fecha.
df_r2['posted'] = df_r2['posted'].str.replace('Posted', '').str.replace(',', '').str.strip()
df_r2['posted'] = pd.to_datetime(df_r2['posted'], errors='coerce')

# Se elimina la hora.
df_r2['posted'] = df_r2['posted'].dt.strftime('%Y-%m-%d')

In [66]:
#Se viasualiza los primeros registros del df.
df_r2.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
2,evcentric,http://steamcommunity.com/id/evcentric,,NaN,,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...
3,doctr,http://steamcommunity.com/id/doctr,,2013-10-14,,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,2014-04-15,,211420,35 of 43 people (81%) found this review helpful,True,Git gud


In [67]:
#Se explora la composición del df.
df_r2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25799 entries, 0 to 18309
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      25486 non-null  object
 1   user_url     25486 non-null  object
 2   funny        25458 non-null  object
 3   posted       20824 non-null  object
 4   last_edited  25458 non-null  object
 5   item_id      25458 non-null  object
 6   helpful      25458 non-null  object
 7   recommend    25458 non-null  object
 8   review       25458 non-null  object
dtypes: object(9)
memory usage: 2.0+ MB


In [68]:
#Se eliminan los nulos.
df_r2.dropna(inplace=True)

Creación columna 'sentiment_analisys'

In [20]:
#Se inicializa el analizador de sentimientos.
sia = SentimentIntensityAnalyzer()

In [21]:
# Función para asignar el valor de sentimiento.
def get_sentiment(row):
    #Se verifica si es una cadena de texto.
    if isinstance(row, str):
        #Se verifica si la cadena está vacía o contiene solo espacios.
        if not row.strip():
            # Valor neutral si el registro está vacío
            return 1
        #Se calculan los puntajes de sentimiento usando sia.polarity_scores .
        sentiment_scores = sia.polarity_scores(row)
        #Se obtiene el puntaje compuesto.
        compound_score = sentiment_scores['compound']
        #Se evalúa el puntaje compuesto para determinar el sentimiento.
        if compound_score >= 0.05:
            return 2  # Sentimiento positivo
        elif compound_score <= -0.05:
            return 0  # Sentimiento negativo
        else:
            return 1  # Sentimiento neutro
    else:
        return 1  # Valor neutral si no es una cadena de texto

In [69]:
#Se aplica la función a cada fila del DataFrame y se crea la nueva columna 'sentiment_analysis'.
df_r2['sentiment_analysis'] = df_r2['review'].apply(get_sentiment)

In [70]:
#Se elimina la columna 'review'.
del df_r2['review']
# O puedes usar df_r2.drop('review', axis=1, inplace=True) para eliminarla también

In [71]:
#Se vizualizan los primeros registros del df para ver como quedó.
df_r2.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-11-05,,1250,No ratings yet,True,2
1,js41637,http://steamcommunity.com/id/js41637,,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,True,2
3,doctr,http://steamcommunity.com/id/doctr,,2013-10-14,,250320,2 of 2 people (100%) found this review helpful,True,2
4,maplemage,http://steamcommunity.com/id/maplemage,3 people found this review funny,2014-04-15,,211420,35 of 43 people (81%) found this review helpful,True,1
5,Wackky,http://steamcommunity.com/id/Wackky,,2014-05-05,,249130,7 of 8 people (88%) found this review helpful,True,1


In [72]:
#Se explora la composición del df.
df_r2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20565 entries, 0 to 25467
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             20565 non-null  object
 1   user_url            20565 non-null  object
 2   funny               20565 non-null  object
 3   posted              20565 non-null  object
 4   last_edited         20565 non-null  object
 5   item_id             20565 non-null  object
 6   helpful             20565 non-null  object
 7   recommend           20565 non-null  object
 8   sentiment_analysis  20565 non-null  int64 
dtypes: int64(1), object(8)
memory usage: 1.6+ MB


Teniendo en cuenta las funciones a realizar para el consumo de la API, solo se concideran importantes las columnas 'user_id', 'posted', 'item_id', recomend' y sentiment_analisys'.

In [73]:
df_r2=df_r2.drop(['user_url', 'funny','last_edited', 'helpful'],axis=1)

In [53]:
#Se viasualizan los primeros registros del df.
df_r2.head()

,user_id,posted,item_id,recommend,sentiment_analysis
0,76561197970982479,2011-11-05,1250,True,2
1,js41637,2014-06-24,251610,True,2
3,doctr,2013-10-14,250320,True,2
4,maplemage,2014-04-15,211420,True,1
5,Wackky,2014-05-05,249130,True,1


### PKL

In [75]:
#Se guarda el df procesado en formato PKL.
pkl_file = open('review.pkl', 'ab')
pickle.dump(df_r2, pkl_file)
pkl_file.close()